In [ ]:
import os
import time
from datetime import date, datetime

import geopandas as gpd
import googlemaps
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from shapely.geometry import Point
from sqlalchemy import create_engine
import time
from pathlib import Path

from mods.config import GSEARCH_CITY_CODE
from mods import gmap

load_dotenv()

True

In [ ]:
def e_get_gdf(folder, file_name):
    # E_取得gml檔案，並轉換成gdf物件
    folder = Path(r"C:\Users\add41\Documents\Data_Engineer\Project\example_data")
    path = folder / file_name

    gdf = gpd.read_file(path)

    return gdf

In [ ]:
def get_search_data_dict():
    return {
        "today":date.today().strftime("%Y/%m/%d"),
        "city_list":list(GSEARCH_CITY_CODE.keys()),
        "city_eng_list":list(GSEARCH_CITY_CODE.values()),
        "city_dict":GSEARCH_CITY_CODE
    }

In [2]:
# E_取得gml檔案，並轉換成gdf物件
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data"
file = "COUNTY_MOI_1140318.gml"
target = os.path.join(folder, file)

gdf = gpd.read_file(target)

# 列出目標縣市（台北市被包含在新北市中），並設定英文代號
today = date.today().strftime("%Y/%m/%d")
target_city = ["新北市", "桃園市", "臺中市", "臺南市", "高雄市"]
city_eng = ["TPE", "TYU", "TCH", "TNA", "KSH"]
target_index = []
city_dict = {
    "新北市":"TPE",
    "桃園市":"TYU",
    "臺中市":"TCH",
    "臺南市":"TNA",
    "高雄市":"KSH"
}

# 將目標縣市的索引加入列表中
for city in target_city:
    target_index.append(gdf[gdf["名稱"] == city].index)

In [3]:
def gmap_nearby_search(key, lat, lon, radius, keyword):
    """提供api_key，經緯度和關鍵字，回傳搜尋結果的列表，資訊包括名稱、place_id和地址"""
    # 使用gmap連線並搜尋，取得搜尋結果
    gmaps = googlemaps.Client(key=key)
    search_result = gmaps.places_nearby(location=(lat, lon), radius=radius, keyword=keyword)
    result = search_result.get("results", [])
    page = 1
    
    # 取出第一頁搜尋結果的名稱、place_id和地址，存成字典後加入list
    result_data = []
    for place in result:
        place_dict = {
            "name":place.get("name", None),
            "place_id":place.get("place_id", None),
            "buss_status":place.get("business_status", None)
        }
        result_data.append(place_dict)


    # 進入迴圈判斷是否有下一頁，如果沒有直接停止，若有則迴圈
    time.sleep(2.5)
    next_page = search_result.get("next_page_token", None)
    
    while next_page:
        page += 1
        max_tries = 3
        for tries in range(1, max_tries+1):
            try:
                print(f"正在取得第{page}頁資訊...")
                time.sleep(2.5)
                next_page_search = gmaps.places_nearby(page_token=next_page)
                next_page_result = next_page_search.get("results", [])
                for place in next_page_result:
                    place_dict = {
                        "name":place.get("name", None),
                        "place_id":place.get("place_id", None),
                        "buss_status":place.get("business_status", None)
                    }
                    result_data.append(place_dict)
                time.sleep(2.5)
                next_page = next_page_search.get("next_page_token", None)
                break

            except Exception as e:
                if tries >= max_tries:
                    print("已達最大嘗試次數")
                    next_page = None
                    break
                else:
                    print(f"第{tries}次嘗試失敗：{e}\n等待3秒後重試...")
                    time.sleep(3)
    
    return result_data

In [ ]:
## E_透過gmap API遍尋六都資料

# 定義關鍵字列表和對應的英文名 # "寵物美容", "寵物餐廳",
keyword_list = ["寵物用品"]
keyword_dict = {
    "寵物美容":"salon",
    "寵物餐廳":"restaurant",
    "寵物用品":"supplies"
}

# 根據存取的索引列表去取出每一個縣市的多邊形資料
for keyword in keyword_list:
    metadate = []
    for idx in target_index:
        name = city_dict[gdf.loc[idx]["名稱"].values[0]]
        city_geo_data = gdf.loc[idx].geometry.values[0]

        # 取得四極點
        min_x, min_y, max_x, max_y = city_geo_data.bounds

        # 設定搜尋半徑及步長
        radius_m = 3000
        step_m = 3000

        # 將距離換算成經緯度
        lat_step = step_m / 111000
        lon_step = step_m / (111000 * np.cos(np.radians((min_y + max_y) / 2)))

        # 產生在範圍內的所有座標點
        lat_point = np.arange(min_y, max_y, lat_step)
        lon_point = np.arange(min_x, max_x, lon_step)

        print(f"開始搜尋{name}的{keyword}資料")
        print(f"lat個數：{len(lat_point)}")
        print(f"lon個數：{len(lon_point)}")

        # 將所有取得的經緯度組合成所有座標點，並形成一個list
        points = []

        for lat_p in lat_point:
            for lon_p in lon_point:
                loc = (lon_p, lat_p)
                points.append(loc)

        print(f"座標點個數：{len(points)}")

        # 開始進行gmap資料爬取
        data = []
        key = os.environ.get("GMAP_KEY1")
        count = 1

        for loc in points:
            loc_p = Point(loc[0], loc[1])

            if city_geo_data.contains(loc_p):
                lat = loc[1]
                lon = loc[0]
                result_list = gmap.gmap_nearby_search(key=key, lat=lat, lon=lon, radius=radius_m, keyword=keyword)
                for i in result_list:
                    data.append(i)

                print(f"完成{name}的{keyword}的第{count}/{len(points)}個座標點（{round(float(loc[1]), 7)}, {round(float(loc[0]), 7)}）的搜尋，共有{len(result_list)}筆店家資料")
                count += 1
                time.sleep(1)

        city_metadata = {
            "city":name,
            "search_radius":radius_m,
            "step":step_m,
            "coord_count":count,
            "data_count":len(data),
            "type":keyword_dict[keyword],
            "update_date":today
        }
        metadate.append(city_metadata)
        print(f"完成{name}的{keyword}資料搜尋，總共搜尋{count}個座標點")

        # 將結果轉換成df並建立更新時間欄位
        df = pd.DataFrame(data=data)
        df["update_time"] = datetime.now().strftime("%Y/%m/%d %H:%M:%S")

        # 將檔案儲存為csv
        save = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\gmap_full_search"
        file_name = f"gmap_{keyword_dict[keyword]}_{name}.csv"
        save_path = os.path.join(save, file_name)
        df.to_csv(save_path, index=False, encoding="utf-8")

        print(f"已完成{name}資料儲存")
        time.sleep(3)


    # E_儲存元數據報告
    username = os.getenv("MYSQL_USERNAME")
    password = os.getenv("MYSQL_PASSWORD")
    target_ip = os.getenv("MYSQL_IP")
    target_port = int(os.getenv("MYSQL_PORTT"))
    db_name = os.getenv("MYSQL_DB_NAME")

    engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

    sql = "SELECT * FROM gmap_record;"

    df_record = pd.read_sql(sql, engine)

    # 將本次報告與原有紀錄合併
    df_meta = pd.DataFrame(metadate)
    df_meta["type"] = keyword_dict[keyword]

    df_record = pd.concat([df_record, df_meta], ignore_index=True)
    df_record.drop_duplicates(keep="first", inplace=True)


    # 按照日期進行儲存
    today_save = date.today().strftime("%Y%m%d")

    meta_save = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\gmap_full_search"
    meta_name = f"{today_save}_{keyword_dict[keyword]}_record.csv"
    save_path = os.path.join(meta_save, meta_name)

    df_record.to_csv(save_path, index=False, encoding="utf-8")
    df_record.to_sql(name="gmap_record", con=engine, index=False, if_exists="replace")

開始搜尋TPE的寵物用品資料
lat個數：24
lon個數：25
座標點個數：600
完成TPE的寵物用品的第1/600個座標點（24.7002191, 121.4914128）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第2/600個座標點（24.7002191, 121.5212306）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第3/600個座標點（24.7272462, 121.4615949）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第4/600個座標點（24.7272462, 121.4914128）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第5/600個座標點（24.7272462, 121.5212306）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第6/600個座標點（24.7272462, 121.5510484）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第7/600個座標點（24.7272462, 121.5808662）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第8/600個座標點（24.7542732, 121.4317771）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第9/600個座標點（24.7542732, 121.4615949）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第10/600個座標點（24.7542732, 121.4914128）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第11/600個座標點（24.7542732, 121.5212306）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第12/600個座標點（24.7542732, 121.5510484）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第13/600個座標點（24.7542732, 121.5808662）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第14/600個座標點（24.7813002, 121.4615949）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第15/600個座標點（24.7813002, 121.4914128）的搜尋，共有0筆店家資料
完成TPE的寵物用品的第16/600個座標點（24.7813002, 121.5212306）的搜尋，共有0筆店家資料
完成TPE的